In [528]:
#libs
import pymongo
from ucimlrepo import fetch_ucirepo
import random

### Подготовка данных для загрузки в базу

In [529]:
wine_quality = fetch_ucirepo(id=186) 
df = wine_quality.data.original
df.insert(0, 'bottle', list(range(len(df))))
data = df.to_dict('records')


### Подключение и создание базы

In [530]:
# Connect to mongo
client = pymongo.MongoClient("mongodb://localhost:27017/")

# Create table
db = client["wine"]

# Create collection 
collection = db["bottles"]

# Create / Загрузка датасета 

In [531]:
collection.insert_many(data)
collection.count_documents({}) == len(df)

True

In [532]:
db.list_collection_names()

['bottles']

Видно, что была создана колекция bottles и в ней столько же документов, что было в датасете

# Read

In [533]:
df

,bottle,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,color
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,red
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,red
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,red
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,white
6493,6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,white
6494,6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,white
6495,6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,white


In [534]:
collection.find_one({'color' : 'red', 'quality': 7})

{'_id': ObjectId('65f35b9219f48c60f3b03685'),
 'bottle': 7,
 'fixed_acidity': 7.3,
 'volatile_acidity': 0.65,
 'citric_acid': 0.0,
 'residual_sugar': 1.2,
 'chlorides': 0.065,
 'free_sulfur_dioxide': 15.0,
 'total_sulfur_dioxide': 21.0,
 'density': 0.9946,
 'pH': 3.39,
 'sulphates': 0.47,
 'alcohol': 10.0,
 'quality': 7,
 'color': 'red'}

In [535]:
for bottle in collection.find({'color' : 'white', 'quality': {"$gt": 8}, 'alcohol': {"$lt" : 14}}):
    print(bottle)

{'_id': ObjectId('65f35b9219f48c60f3b03fc3'), 'bottle': 2373, 'fixed_acidity': 9.1, 'volatile_acidity': 0.27, 'citric_acid': 0.45, 'residual_sugar': 10.6, 'chlorides': 0.035, 'free_sulfur_dioxide': 28.0, 'total_sulfur_dioxide': 124.0, 'density': 0.997, 'pH': 3.2, 'sulphates': 0.46, 'alcohol': 10.4, 'quality': 9, 'color': 'white'}
{'_id': ObjectId('65f35b9219f48c60f3b03ff1'), 'bottle': 2419, 'fixed_acidity': 6.6, 'volatile_acidity': 0.36, 'citric_acid': 0.29, 'residual_sugar': 1.6, 'chlorides': 0.021, 'free_sulfur_dioxide': 24.0, 'total_sulfur_dioxide': 85.0, 'density': 0.98965, 'pH': 3.41, 'sulphates': 0.61, 'alcohol': 12.4, 'quality': 9, 'color': 'white'}
{'_id': ObjectId('65f35b9219f48c60f3b03ff8'), 'bottle': 2426, 'fixed_acidity': 7.4, 'volatile_acidity': 0.24, 'citric_acid': 0.36, 'residual_sugar': 2.0, 'chlorides': 0.031, 'free_sulfur_dioxide': 27.0, 'total_sulfur_dioxide': 139.0, 'density': 0.99055, 'pH': 3.28, 'sulphates': 0.48, 'alcohol': 12.5, 'quality': 9, 'color': 'white'}
{

In [536]:
collection.count_documents({'color' : 'white'})

4898

In [537]:
for bottle in collection.find({'color' : 'red', 'quality': {"$gt": 7}}).sort('alcohol'):
    print(bottle['bottle'], bottle['alcohol'], bottle['quality'])

440 9.8 8
1403 10.0 8
495 11.0 8
498 11.0 8
1449 11.3 8
1549 11.4 8
481 11.7 8
1202 11.7 8
1090 11.8 8
1061 12.5 8
278 12.6 8
828 12.7 8
267 12.8 8
390 12.9 8
1120 13.1 8
455 13.4 8
588 14.0 8
1269 14.0 8


In [538]:
collection.distinct("color")

['red', 'white']

In [539]:
collection.distinct("quality")

[3, 4, 5, 6, 7, 8, 9]

#  Update

In [540]:
query = {"quality" : 9}
new_values = {"$set": {"quality" : 10}}
collection.update_many(query, new_values)

In [541]:
collection.distinct("quality")

[3, 4, 5, 6, 7, 8, 10]

In [542]:
query = {"bottle" : 9}
new_values = {"$set": {"quality" : 10, 'alcohol' : 0}}
collection.update_one(query, new_values)

In [543]:
collection.find_one({'bottle' : 9})

{'_id': ObjectId('65f35b9219f48c60f3b03687'),
 'bottle': 9,
 'fixed_acidity': 7.5,
 'volatile_acidity': 0.5,
 'citric_acid': 0.36,
 'residual_sugar': 6.1,
 'chlorides': 0.071,
 'free_sulfur_dioxide': 17.0,
 'total_sulfur_dioxide': 102.0,
 'density': 0.9978,
 'pH': 3.35,
 'sulphates': 0.8,
 'alcohol': 0,
 'quality': 10,
 'color': 'red'}

# Delete

In [544]:
collection.insert_one({'color': 'green', "quality" : 1})

In [545]:
collection.find_one({'color': 'green'})

{'_id': ObjectId('65f35b9219f48c60f3b04fdf'), 'color': 'green', 'quality': 1}

In [546]:
collection.delete_one({'color': 'green'})

In [547]:
collection.find_one({'color': 'green'})

In [548]:
collection.delete_many({'quality' : 3})

In [549]:
collection.distinct("quality")

[4, 5, 6, 7, 8, 10]

# Indexes

Нагенерим случайных документов для наглядности работы индексов

In [550]:
random_data= []
for i in range(1000000):
    random_data.append({
      "bottle": 6496 + i,
      "density": random.random(),
      "pH": random.gauss(5, 2),
      "sulphates": random.random(),
      "alcohol" : random.gauss(10, 5),
      "quality" : random.randint(1, 10),
      "color" : random.choice(["white", "red", "pink"])
  })

In [551]:
collection.insert_many(random_data)
collection.count_documents({})

1006467

In [552]:
%%time
collection.find({"quality" : 6}).explain()

CPU times: user 1.93 ms, sys: 1.39 ms, total: 3.31 ms
Wall time: 570 ms


{'explainVersion': '1',
 'queryPlanner': {'namespace': 'wine.bottles',
  'indexFilterSet': False,
  'parsedQuery': {'quality': {'$eq': 6}},
  'queryHash': '9178AA23',
  'planCacheKey': '9178AA23',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'quality': {'$eq': 6}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 103243,
  'executionTimeMillis': 568,
  'totalKeysExamined': 0,
  'totalDocsExamined': 1006467,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'quality': {'$eq': 6}},
   'nReturned': 103243,
   'executionTimeMillisEstimate': 53,
   'works': 1006468,
   'advanced': 103243,
   'needTime': 903224,
   'needYield': 0,
   'saveState': 1006,
   'restoreState': 1006,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 1006467},
  'allPlansExecution': []},
 'command': {'find': '

In [553]:
collection.create_index([('quality', 1)])

'quality_1'

In [554]:
%%time
collection.find({"quality" : 6}).explain()

CPU times: user 1.89 ms, sys: 1.77 ms, total: 3.66 ms
Wall time: 151 ms


{'explainVersion': '1',
 'queryPlanner': {'namespace': 'wine.bottles',
  'indexFilterSet': False,
  'parsedQuery': {'quality': {'$eq': 6}},
  'queryHash': '9178AA23',
  'planCacheKey': 'B711904F',
  'maxIndexedOrSolutionsReached': False,
  'maxIndexedAndSolutionsReached': False,
  'maxScansToExplodeReached': False,
  'winningPlan': {'stage': 'FETCH',
   'inputStage': {'stage': 'IXSCAN',
    'keyPattern': {'quality': 1},
    'indexName': 'quality_1',
    'isMultiKey': False,
    'multiKeyPaths': {'quality': []},
    'isUnique': False,
    'isSparse': False,
    'isPartial': False,
    'indexVersion': 2,
    'direction': 'forward',
    'indexBounds': {'quality': ['[6, 6]']}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 103243,
  'executionTimeMillis': 149,
  'totalKeysExamined': 103243,
  'totalDocsExamined': 103243,
  'executionStages': {'stage': 'FETCH',
   'nReturned': 103243,
   'executionTimeMillisEstimate': 23,
   'works': 103244,
   'advan

При использовании индексов наблюдается ускорение по времени работы и количеству просмотренных документов

In [555]:
collection.drop()